### Постановка задачи
Построить простую модель энергопотребления здания по среднему значению, 
оценить эффективность модели через метрику
\begin{equation}
RMSLE = {\sqrt{\sum_{i=1}^{n}{(log(p_i+1) - log(a_i+1))^2} \over n}}.
\end{equation}
* n - число наблюдений
* log - натуральный логарифм
* p_i - вычисленное значение метрики
* a_i - заданное значение метрики

Данные: http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/


### Загрузка библиотек

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams 
rcParams['figure.figsize'] = 16, 8

### Загрузка данных
Дополнительно сразу отсечем пустые дни и выделим час из значения времени

In [2]:
energy_0 = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz")
energy_0 = energy_0[energy_0["meter_reading"] > 0]
energy_0["timestamp"] = pd.to_datetime(energy_0["timestamp"])
energy_0["hour"] = energy_0["timestamp"].dt.hour
print (energy_0.head())

      building_id  meter           timestamp  meter_reading  hour
704             0      0 2016-01-30 08:00:00        43.6839     8
725             0      0 2016-01-31 05:00:00        37.5408     5
737             0      0 2016-01-31 17:00:00        52.5571    17
2366            0      0 2016-04-08 14:00:00        59.3827    14
2923            0      0 2016-05-01 19:00:00       448.0000    19


### Разделение данных на обучение и проверку
Выделим 20% всех данных на проверку, остальные оставим на обучение

In [3]:
energy_0_train, energy_0_test = train_test_split(energy_0, test_size=0.2)
print (energy_0_train.head())

      building_id  meter           timestamp  meter_reading  hour
4068            0      0 2016-06-18 12:00:00        298.279    12
4384            0      0 2016-07-01 16:00:00        251.865    16
4177            0      0 2016-06-23 01:00:00        247.769     1
6555            0      0 2016-09-30 03:00:00        250.500     3
8420            0      0 2016-12-16 20:00:00        232.753    20


### Создадим модели
Среднее и медианное значение потребление энергии по часам

In [4]:
energy_0_train_hours = energy_0_train.groupby("hour")
energy_0_train_averages = pd.DataFrame(
    {"Среднее": energy_0_train_hours.mean()["meter_reading"],
     "Медиана": energy_0_train_hours.median()["meter_reading"]})
print (energy_0_train_averages)

         Среднее   Медиана
hour                      
0     236.703761  239.5790
1     237.372915  240.2610
2     240.557677  243.6740
3     240.983943  244.3570
4     241.414262  247.7690
5     236.997593  245.0390
6     234.571969  243.6740
7     241.468538  246.4040
8     241.062656  238.5550
9     233.958864  233.7770
10    234.656059  238.8960
11    231.981276  241.6260
12    233.238415  241.9675
13    236.806238  242.9910
14    238.019696  242.3090
15    234.273144  241.6260
16    237.276911  242.9910
17    235.487056  240.2610
18    238.640114  240.2610
19    235.368254  237.1895
20    235.225446  236.1660
21    236.985711  237.5310
22    237.136026  238.2140
23    240.048596  239.5790


### Функция проверки модели
\begin{equation}
RMSLE = {\sqrt{\sum_{i=1}^{n}{(log(p_i+1) - log(a_i+1))^2} \over n}}.
\end{equation}

Для вычисления метрики создадим шесть новых столбцов в тестовом наборе данных: с логарифмом значения метрики, предсказанием по среднему и по медиане, а также с квадратом разницы предсказаний и логарифма значения. Последний столбец добавим, чтобы сравнить предсказание с его отсутствием - нулями в значениях.

In [5]:
def calculate_model (x):
    meter_reading_log = np.log(x.meter_reading + 1)
    meter_reading_mean = np.log(energy_0_train_averages["Среднее"][x.hour] + 1)
    meter_reading_median = np.log(energy_0_train_averages["Медиана"][x.hour] + 1)
    x["meter_reading_mean_q"] = (meter_reading_log - meter_reading_mean)**2
    x["meter_reading_median_q"] = (meter_reading_log - meter_reading_median)**2
    x["meter_reading_zero_q"] = (meter_reading_log)**2
    return x

energy_0_test = energy_0_test.apply(calculate_model,
                                    axis=1, result_type="expand")
print (energy_0_test.head())

      building_id  meter           timestamp  meter_reading  hour  \
4322            0      0 2016-06-29 02:00:00        240.261     2   
7991            0      0 2016-11-28 23:00:00        228.658    23   
7839            0      0 2016-11-22 15:00:00        188.387    15   
4528            0      0 2016-07-07 16:00:00        245.039    16   
4773            0      0 2016-07-17 21:00:00        301.009    21   

      meter_reading_mean_q  meter_reading_median_q  meter_reading_zero_q  
4322              0.000002                0.000197             30.094872  
7991              0.002343                0.002158             29.556524  
7839              0.047069                0.061369             27.497360  
4528              0.001028                0.000070             30.310421  
4773              0.056761                0.055676             32.609317  


Теперь остается просуммировать квадраты расхождений, разделить на количество значений и извлечь квадратный корень

In [7]:
energy_0_test_median_rmsle = np.sqrt(energy_0_test["meter_reading_median_q"].sum() / len(energy_0_test))
energy_0_test_mean_rmsle = np.sqrt(energy_0_test["meter_reading_mean_q"].sum() / len(energy_0_test))
energy_0_test_zero_rmsle = np.sqrt(energy_0_test["meter_reading_zero_q"].sum() / len(energy_0_test))
print ("Качество медианы:", energy_0_test_median_rmsle)
print ("Качество среднего:", energy_0_test_mean_rmsle)
print ("Качество нуля:", energy_0_test_zero_rmsle)

Качество медианы: 0.2250585760095414
Качество среднего: 0.22370587337686046
Качество нуля: 5.468987993788107
